In [ ]:
45Nskx66IUnIa2AIWH1tPPi1zVvArbMfhg5wuXUo

In [31]:
import pandas as pd
from tqdm import tqdm
import os
import json
import random
import re
from collections import Counter


In [15]:
45/60 * 12.21 * 9

82.4175

In [ ]:
# df = pd.read_excel(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\Annotation_Round.xlsx", sheet_name="Test_Qualtrics", header=None)

In [4]:
df = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\Annotation_Round - Sheet_Round_2.csv", header=None)

In [5]:
df

,0,1,2,3,4,5,6,7,8
0,There should be a general vaccination requirem...,European agricultural subsidies must be signif...,NaN,1,Unrelated,NaN,NaN,NaN,100 - Surface -> Mark the ones Betuel used in ...
1,Electricity is to be permanently generated fro...,Recipients of unemployment benefits shall cont...,NaN,2,NaN,NaN,NaN,NaN,NaN
2,"To combat terrorism, the armed forces should a...",The state should financially support the insta...,NaN,3,NaN,NaN,NaN,NaN,NaN
3,There should be an EU-wide smoking ban in the ...,We support stricter measures for pollution cau...,NaN,4,NaN,NaN,NaN,NaN,NaN
4,"In the European Union, every adult has to be a...",The conversion of rental apartments into condo...,NaN,5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
345,We advocate for higher Erasmus+ scholarships f...,The Erasmus program provides sufficient resour...,NaN,96,NaN,NaN,NaN,NaN,NaN
346,We vote for the exclusive promotion of ecologi...,The European Union should drastically reduce i...,NaN,97,NaN,NaN,NaN,NaN,NaN
347,We support equipping police forces with tasers...,The consequences of tasers' use have not been ...,NaN,98,NaN,NaN,NaN,NaN,NaN
348,If the government provides financial aid to co...,Further tightening of decision-making processe...,NaN,99,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create an empty list to store the modified rows
modified_rows = []
modified_rows.append("[[AdvancedFormat]]")
modified_rows.append("[[Block:MC Block]]")

In [6]:
def process_columns(rows, id):
    text1 = rows[0]
    text2 = rows[1]
    
    text1 = text1.replace('"', '')
    text2 = text2.replace('"', '')

    question = f"<strong>Q{id}.</strong>" + "<br>"
    
    if "Text 1:" not in text1:
        text1 = "<strong>Text 1:</strong> " + text1.strip() + "<br>"
    elif "Text 1:" in text1:
        text1 = text1.strip()
        text1 = text1.replace('Text 1:', '<strong>Text 1:</strong> ')
        text1 += "<br>"
    if "Text 2:" not in text2:
        text2 = "<strong>Text 2:</strong> " + text2.strip()
    elif "Text 2:" in text2:
        text2 = text2.strip()
        text2 = text2.replace('Text 2:', '<strong>Text 2:</strong> ')
        text2 += "<br>"

    # merged += "\n" + text1 + "\n" + text2
    return question, text1, text2

In [39]:
def process_columns_leftovers(participant_to_survey, version, id, counter_question):
    text1 = participant_to_survey[version][id].split("\n\n")[0]
    text2 = participant_to_survey[version][id].split("\n\n")[1]

    question = f"<strong>Q{counter_question}.</strong>" + "<br>"
    
    if "Text 1:" not in text1:
        text1 = "<strong>Text 1:</strong>" + text1.strip() + "<br>"
    elif "Text 1:" in text1:
        text1 = text1.strip()
        text1 = text1.replace('Text 1:', '<strong>Text 1:</strong> ')
        text1 += "<br>"
    if "Text 2:" not in text2:
        text2 = "<strong>Text 2:</strong>" + text2.strip()
    elif "Text 2:" in text2:
        text2 = text2.strip()
        text2 = text2.replace('Text 2:', '<strong>Text 2:</strong> ')
        text2 += "<br>"

    # merged += "\n" + text1 + "\n" + text2
    return question, text1, text2

In [52]:
# Iterate through each row in the original DataFrame
for id, row in df.iterrows():
    modified_rows.append("[[Question:MC:SingleAnswer:Horizontal]] ")
    # Add the original row
    new_question = process_columns(row, id)
    for el in new_question:
        modified_rows.append(el)
    # Add the additional rows
    modified_rows.append("[[Choices]]")
    modified_rows.append("Unrelated")
    modified_rows.append("Consistent")
    modified_rows.append("Indirect inconsistency")
    modified_rows.append("Factual inconsistency")
    modified_rows.append("Surface contradiction")
    modified_rows.append("[[PageBreak]]")

# Create a new DataFrame from the modified rows
output_df = pd.DataFrame(modified_rows)

# Save the new DataFrame to an Excel file
#output_df.to_excel(output_file, index=False)

#print(f"Modified data written to {output_file}")


In [7]:
output_df

,0
0,[[AdvancedFormat]]
1,[[Block:MC Block]]
2,[[Question:MC:SingleAnswer:Horizontal]]
3,<strong>Text 1:</strong> The first degree shou...
4,<strong>Text 2:</strong> Scholarship should be...
...,...
3497,Consistent
3498,Indirect inconsistency
3499,Factual inconsistency
3500,Surface contradiction


In [19]:
output_df.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\Round_1.txt", sep='\n', index=False)

### Add randomization

In [2]:
import random

In [9]:
# Parameters
num_samples = 350
num_versions = 10
samples_per_version = 175
annotations_per_sample = 5
version_to_sample = {f"V_{i+1}": [] for i in range(num_versions)}

# Initialize data structures
samples = list(range(0, num_samples))
# Duplicate each element 5 times
duplicated_samples = [sample for sample in samples for _ in range(5)]

for version in version_to_sample:
    ids = random.sample(duplicated_samples, k=samples_per_version)
    version_to_sample[version] = ids
    for el in ids:
        duplicated_samples.remove(el)

In [10]:
# Ensure constraints are met
for version, assigned_samples in version_to_sample.items():
    assert len(assigned_samples) == samples_per_version, f"{version} has incorrect sample count"

In [11]:
duplicated_samples

[]

Load ready files (spreading V8 over other participants)

In [3]:
participants = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10']

In [4]:
dfs_participants = dict()

In [7]:
for participant in participants:
    dfs_participants[participant] = pd.read_csv(rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\{participant.capitalize()}.csv")

In [ ]:
answers = [v1, v2, v3, v4, v5, v6, v8, v9, v10]
df_to_participant = {v1.to_string(): 'v1', v2.to_string(): 'v2', v3.to_string(): 'v3', v4.to_string(): 'v4', v5.to_string(): 'v5', v6.to_string(): 'v6', v8.to_string(): 'v8', v9.to_string(): 'v9', v10.to_string(): 'v10'}

sample_to_participants = dict()
sample_to_responses = dict()
for i, df in enumerate(answers):
    print(i+1)
    for column in df.columns:
        string = df[column].loc[0]
        if re.match(r"^Q\d+", string):
            try:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n\n")[2]
            except:
                txt1, txt2 = string.split("\n\n")[1], string.split("\n")[-1]
            label = df[column].iloc[-1]
            try:
                sample_to_responses[txt1 + "\n\n" + txt2].append(label)
                sample_to_participants[txt1 + "\n\n" + txt2].append(df_to_participant[df.to_string()])
            except:
                sample_to_responses[txt1 + "\n\n" + txt2] = [label]
                sample_to_participants[txt1 + "\n\n" + txt2] = [df_to_participant[df.to_string()]]
                
            
          

In [17]:
dfs_participants['v7']

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,QID4312,QID4313,QID4314,QID4315,QID4316,QID4317,QID4318,QID4319,QID4320,SC0
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Please explain the reasoning behind your answe...,Q169.\n\nText 1: We support imposing stricter ...,Q170.\n\nText 1: A minimum wage law should be ...,Q171.\n\nText 1: We demand unrestricted asylum...,Q172.\n\nText 1: We must protect our indigenou...,Q173.\n\nText 1: We advocate for decreasing th...,Please explain the reasoning behind your answe...,Q174.\n\nText 1: We want broad support for agr...,Q175.\n\nText 1: We support imposing stricter ...,Score
1,"{""ImportId"":""startDate"",""timeZone"":""Europe/Ber...","{""ImportId"":""endDate"",""timeZone"":""Europe/Berlin""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""Europe/...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID4312_TEXT""}","{""ImportId"":""QID4313""}","{""ImportId"":""QID4314""}","{""ImportId"":""QID4315""}","{""ImportId"":""QID4316""}","{""ImportId"":""QID4317""}","{""ImportId"":""QID4318_TEXT""}","{""ImportId"":""QID4319""}","{""ImportId"":""QID4320""}","{""ImportId"":""SC_6DoUpyPQIBjSKQS""}"
2,2025-01-13 19:49:42,2025-01-14 22:53:42,IP Address,95.91.223.2,100,97440,True,2025-01-14 22:53:43,R_8pW4SIkgzvudCJj,NaN,...,Text 1 advocates for making contraceptives wid...,Indirect inconsistency,Surface contradiction,Surface contradiction,Consistent,Consistent,Text 1 suggests a position aimed at reducing t...,Factual inconsistency,Indirect inconsistency,3


In [21]:
json_to_participants['v7'] = []
for column in dfs_participants['v7']: 
    string = dfs_participants['v7'][column].iloc[0]
    if re.match(r"^Q\d+", string):
        try:
            txt1, txt2 = string.split("\n\n")[1], string.split("\n\n")[2]
        except:
            txt1, txt2 = string.split("\n\n")[1], string.split("\n")[-1]
        
        res = "\n\n".join([txt1, txt2])
        json_to_participants['v7'].append(res)
        

In [11]:
json_to_participants = dict()
for participant in participants:
    if participant != 'v7':
        with open(rf'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\part_2_leftovers\{participant}.json', 'r') as f:
            json_to_participants[participant] = json.load(f)

In [28]:
sample_to_not_answered_participant = dict()
for sample in json_to_participants['v8']:
    sample_to_not_answered_participant[sample] = []
    for participant in json_to_participants:
        if sample not in json_to_participants[participant]:
            sample_to_not_answered_participant[sample].append(participant)
            

In [30]:
len(sample_to_not_answered_participant)

41

In [39]:
counter_not_answered = Counter()
samples_to_preserve = []
for sample in sample_to_not_answered_participant:
    lst = Counter(sample_to_not_answered_participant[sample])
    if 'v5' in lst and 'v3' in lst:
        samples_to_preserve.append(sample)
    # counter_not_answered.update(Counter(sample_to_not_answered_participant[sample]))

In [43]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\part_2_leftovers\v7.json', 'w') as f:
    json.dump(json_to_participants['v7'], f)

In [33]:
counter_not_answered

Counter({'v3': 40,
         'v5': 40,
         'v10': 39,
         'v4': 38,
         'v9': 38,
         'v1': 37,
         'v6': 36,
         'v2': 35,
         'v7': 26})

In [ ]:
ask v5 or v3 equally

In [44]:
12.21/60

0.20350000000000001

In [ ]:
no 5
 'Text 1: Electricity is to be permanently generated from lignite.\n\nText 2: Recipients of unemployment benefits shall continue to have their benefits reduced if they refuse to take on reasonable employment.': ['v1',
  'v2',
  'v3',
  'v4',
  'v6',
  'v9',
  'v7'],
 
no v3
Text 1: Landlords should be allowed to decide whether they can rent out their apartments as vacation rentals.

Text 2: A certain percentage of new apartments should be required to be social housing.


 'Text 1: Landlords should be allowed to decide whether they can rent out their apartments as vacation rentals.\n\nText 2: A certain percentage of new apartments should be required to be social housing.': ['v1',
  'v2',
  'v4',
  'v5',
  'v6',
  'v9',
  'v10',
  'v7'],

In [29]:
sample_to_not_answered_participant

{'Text 1: Motivated and confident workforce is a crucial success factor for the economy.\n\nText 2: We support limiting the scope of employee influence in decision-making within companies to promote competitiveness.': ['v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v9',
  'v10',
  'v7'],
 'Text 1: We believe that adventure tourism endangers fragile environments.\n\nText 2: High touristic activity is a sign of a thriving local economy.': ['v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v6',
  'v9',
  'v10',
  'v7'],
 'Text 1: Business activities of municipalities should be limited to basic services (e.g. water, gas, electricity, heat).\n\nText 2: We made a donation to our municipality to promote the local arts scene.': ['v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v6',
  'v9',
  'v10',
  'v7'],
 'Text 1: The Nord Stream 2 pipeline should be allowed to operate as planned to provide Germany with sufficient gas supplies from Russia.\n\nText 2: Chinese companies should not be allowed to take contracts for e

In [25]:
json_to_participants['v7'] = json_to_participants['v7'][1:]

In [5]:
dfs_participants['v7'] = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\responses_part_2\V7.csv")

In [6]:
dfs_participants['v7']

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,QID4312,QID4313,QID4314,QID4315,QID4316,QID4317,QID4318,QID4319,QID4320,SC0
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Please explain the reasoning behind your answe...,Q169.\n\nText 1: We support imposing stricter ...,Q170.\n\nText 1: A minimum wage law should be ...,Q171.\n\nText 1: We demand unrestricted asylum...,Q172.\n\nText 1: We must protect our indigenou...,Q173.\n\nText 1: We advocate for decreasing th...,Please explain the reasoning behind your answe...,Q174.\n\nText 1: We want broad support for agr...,Q175.\n\nText 1: We support imposing stricter ...,Score
1,"{""ImportId"":""startDate"",""timeZone"":""Europe/Ber...","{""ImportId"":""endDate"",""timeZone"":""Europe/Berlin""}","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""Europe/...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID4312_TEXT""}","{""ImportId"":""QID4313""}","{""ImportId"":""QID4314""}","{""ImportId"":""QID4315""}","{""ImportId"":""QID4316""}","{""ImportId"":""QID4317""}","{""ImportId"":""QID4318_TEXT""}","{""ImportId"":""QID4319""}","{""ImportId"":""QID4320""}","{""ImportId"":""SC_6DoUpyPQIBjSKQS""}"
2,2025-01-13 19:49:42,2025-01-14 22:53:42,IP Address,95.91.223.2,100,97440,True,2025-01-14 22:53:43,R_8pW4SIkgzvudCJj,NaN,...,Text 1 advocates for making contraceptives wid...,Indirect inconsistency,Surface contradiction,Surface contradiction,Consistent,Consistent,Text 1 suggests a position aimed at reducing t...,Factual inconsistency,Indirect inconsistency,3


Load ready files (correcting Pt.2)

In [3]:
participants = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v8', 'v9', 'v10']
        
participant_to_survey = dict()
for el in participants:
    with open(rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\part_2_leftovers\{el}.json", 'rb') as f:
        res = json.load(f)
    participant_to_survey[el] = dict(enumerate(res))

In [4]:
participant_to_survey_ids = dict()
for el in participants:
    participant_to_survey_ids[el] = list(participant_to_survey[el].keys())

In [19]:
# 18 explanations randomly
# take 20% of all questions
id_explanations = [f"el_{i}" for i in (range(1, 9)) ]

In [20]:
explanation_to_str = {f'{el}': "Please explain the reasoning behind your answer to the previous question. <br> <strong>Note</strong>: explanations should contain meaningful reasoning specific to the question above. Answers like 'A and B are obviously contradictory', 'Text 1 is directly inconsistent with Text 2' and similar will be dismissed." for el in id_explanations}

In [6]:
for version in participant_to_survey_ids: 
    participant_to_survey_ids[version].extend(id_explanations)
    random.shuffle(participant_to_survey_ids[version])

In [15]:
for version in version_to_sample: 
    version_to_sample[version].extend(id_explanations)
    random.shuffle(version_to_sample[version])

In [7]:
# Ensure no version starts from explanation
for version, assigned_samples in version_to_sample.items():
    assert assigned_samples[0] not in id_explanations, f"{version} starts with explanation"

NameError: name 'version_to_sample' is not defined

In [14]:
# Ensure no version starts from explanation
for version, assigned_samples in participant_to_survey_ids.items():
    assert assigned_samples[0] not in id_explanations, f"{version} starts with explanation"

In [13]:
random.shuffle(participant_to_survey_ids['v9'])

In [ ]:
random.shuffle(version_to_sample['V_7'])

In [18]:
for version in version_to_sample: 
    while True:
        random.shuffle(version_to_sample[version])
        if meets_conditions(version_to_sample[version]):
            break

In [16]:
for version in participant_to_survey_ids: 
    while True:
        random.shuffle(participant_to_survey_ids[version])
        if meets_conditions(participant_to_survey_ids[version]):
            break

In [61]:
version_to_sample_split = dict()
for key_name in version_to_sample:
    version_to_sample_split[f'{key_name}_split'] = {key: version_to_sample[key_name][(key - 1) * 25:key * 25] for key in range(1, 8)}


In [39]:
id_explanations = id_explanations[:5]

In [17]:
id_explanations

['el_1', 'el_2', 'el_3', 'el_4', 'el_5', 'el_6', 'el_7', 'el_8']

In [15]:
def meets_conditions(lst):
    # Condition 1: The first element should not start with "el"
    if isinstance(lst[0], str):
        return False
    # Condition 2: No two consecutive elements should start with "el"
    for i in range(len(lst) - 1):
        if isinstance(lst[i], str) and isinstance(lst[i + 1], str):
            return False
    return True

Insert explanations randomly

In [64]:
for version in version_to_sample_split:
    for key in version_to_sample_split[version]:
        version_to_sample_split[version][key].extend(id_explanations)
        
        while True:
            random.shuffle(version_to_sample_split[version][key])
            if meets_conditions(version_to_sample_split[version][key]):
                break
        
    
    

In [65]:
version_to_sample_split

{'V_1_split': {1: [64,
   193,
   80,
   'el_5',
   268,
   323,
   49,
   217,
   344,
   249,
   'el_4',
   37,
   55,
   'el_3',
   221,
   205,
   284,
   97,
   319,
   'el_1',
   102,
   186,
   298,
   109,
   259,
   117,
   250,
   322,
   'el_2',
   182],
  2: [195,
   'el_1',
   343,
   163,
   203,
   276,
   127,
   219,
   290,
   'el_5',
   260,
   'el_3',
   87,
   141,
   105,
   103,
   241,
   'el_4',
   14,
   131,
   227,
   319,
   221,
   'el_2',
   82,
   7,
   79,
   234,
   33,
   72],
  3: [175,
   314,
   'el_1',
   80,
   281,
   311,
   278,
   18,
   188,
   'el_5',
   300,
   189,
   'el_4',
   83,
   127,
   158,
   136,
   166,
   96,
   52,
   114,
   'el_3',
   284,
   44,
   327,
   209,
   81,
   'el_2',
   324,
   305],
  4: [198,
   'el_2',
   149,
   'el_1',
   233,
   29,
   25,
   212,
   47,
   150,
   1,
   41,
   230,
   48,
   171,
   189,
   'el_3',
   106,
   284,
   'el_5',
   57,
   67,
   336,
   243,
   319,
   94,
   'el_4',
   115,

### Create separate versions for each participant

In [66]:
version_to_df = {}
for version in tqdm(version_to_sample_split):
    
    for chunk in version_to_sample_split[version]:
        ids_version = version_to_sample_split[version][chunk]
        print(len(ids_version))
        # df_id = df[df.index.isin(ids_version)]
        modified_rows = []
        modified_rows.append("[[AdvancedFormat]]")
        modified_rows.append("[[Block:MC Block]]")
        # Iterate through each row in the original DataFrame
        counter_question = 1
        for id in ids_version:
            # Add the original row
            if isinstance(id, int):
                modified_rows.append("[[Question:MC:SingleAnswer:Horizontal]]")
                new_question = process_columns(df.loc[id], counter_question)
                counter_question += 1
                for el in new_question:
                    modified_rows.append(el)
                    # Add the additional rows
                modified_rows.append("[[Choices]]")
                modified_rows.append("Unrelated")
                modified_rows.append("Consistent")
                modified_rows.append("Indirect inconsistency")
                modified_rows.append("Factual inconsistency")
                modified_rows.append("Surface contradiction")
            
            elif isinstance(id, str):
                modified_rows.append("[[Question:TextEntry:SingleLine]]")
                new_question = explanation_to_str[str(id)]
                modified_rows.append(new_question)
            
            modified_rows.append("[[PageBreak]]")

        # Create a new DataFrame from the modified rows
        output_df = pd.DataFrame(modified_rows)
        file_path = rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_{version}"
        # Create the directory if it doesn't exist
        if not os.path.exists(file_path):
            os.makedirs(file_path)
            print(f"Directory '{file_path}' created.")
        else:
            print(f"Directory '{file_path}' already exists.")
        output_df.to_csv(file_path+f"\chunk_{chunk}.txt", sep='\n', index=False)
        # version_to_df[version] = output_df
        # print(counter_question)
        
        

100%|██████████| 10/10 [00:00<00:00, 128.07it/s]

30
Directory 'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_V_1_split' already exists.
30
Directory 'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_V_1_split' already exists.
30
Directory 'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_V_1_split' already exists.
30
Directory 'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_V_1_split' already exists.
30
Directory 'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_V_1_split' already exists.
30
Directory 'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_V_1_split' already exists.
30
Directory 'C:\Users\Nursulu_1\Downloads\Con

In [21]:
version_to_df = {}
for version in tqdm(version_to_sample):
    ids_version = version_to_sample[version]
    print(len(ids_version))
    # df_id = df[df.index.isin(ids_version)]
    modified_rows = []
    modified_rows.append("[[AdvancedFormat]]")
    modified_rows.append("[[Block:MC Block]]")
    # Iterate through each row in the original DataFrame
    counter_question = 1
    for id in ids_version:
        # Add the original row
        if isinstance(id, int):
            modified_rows.append("[[Question:MC:SingleAnswer:Horizontal]]")
            new_question = process_columns(df.loc[id], counter_question)
            counter_question += 1
            for el in new_question:
                modified_rows.append(el)
                # Add the additional rows
            modified_rows.append("[[Choices]]")
            modified_rows.append("Unrelated")
            modified_rows.append("Consistent")
            modified_rows.append("Indirect inconsistency")
            modified_rows.append("Factual inconsistency")
            modified_rows.append("Surface contradiction")
           
        elif isinstance(id, str):
            modified_rows.append("[[Question:TextEntry:SingleLine]]")
            new_question = explanation_to_str[str(id)]
            modified_rows.append(new_question)
        
        modified_rows.append("[[PageBreak]]")

    # Create a new DataFrame from the modified rows
    output_df = pd.DataFrame(modified_rows)
    file_path = rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\10_versions_part_2\version_{version}.txt"
    output_df.to_csv(file_path, sep='\n', index=False)
    version_to_df[version] = output_df
    print(counter_question)
    
    

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 213.34it/s]

210
176
210
176
210
176
210
176
210
176
210
176
210
176
210
176
210
176
210
176


Don't need to create a df.

In [23]:
participant_to_survey['v1'][0]

"Text 1: The legal minimum wage should be raised to at least 12 euros.\n\nText 2: Unemployment benefits should be reduced if the recepients don't show enough effort in looking for a job."

In [24]:
text1 = participant_to_survey['v1'][0].split("\n\n")[0]
text2 = participant_to_survey['v1'][0].split("\n\n")[1]

In [28]:
text2

"Text 2: Unemployment benefits should be reduced if the recepients don't show enough effort in looking for a job."

In [40]:
# participant_to_survey_ids

version_to_df = {}
for version in tqdm(participant_to_survey_ids):
    ids_version = participant_to_survey_ids[version]
    print(len(ids_version))
    # df_id = df[df.index.isin(ids_version)]
    modified_rows = []
    modified_rows.append("[[AdvancedFormat]]")
    modified_rows.append("[[Block:MC Block]]")
    # Iterate through each row in the original DataFrame
    counter_question = 1
    for id in ids_version:
        # Add the original row
        if isinstance(id, int):
            modified_rows.append("[[Question:MC:SingleAnswer:Horizontal]]")
            new_question = process_columns_leftovers(participant_to_survey, version, id, counter_question)
            counter_question += 1
            for el in new_question:
                modified_rows.append(el)
                # Add the additional rows
            modified_rows.append("[[Choices]]")
            modified_rows.append("Unrelated")
            modified_rows.append("Consistent")
            modified_rows.append("Indirect inconsistency")
            modified_rows.append("Factual inconsistency")
            modified_rows.append("Surface contradiction")
           
        elif isinstance(id, str):
            modified_rows.append("[[Question:TextEntry:SingleLine]]")
            new_question = explanation_to_str[str(id)]
            modified_rows.append(new_question)
        
        modified_rows.append("[[PageBreak]]")

    # Create a new DataFrame from the modified rows
    output_df = pd.DataFrame(modified_rows)
    file_path = rf"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\9_versions_part_2_leftovers\version_{version}.txt"
    output_df.to_csv(file_path, sep='\n', index=False)
    version_to_df[version] = output_df
    print(counter_question)
    
    

100%|██████████| 9/9 [00:00<00:00, 575.72it/s]

49
42
49
42
48
41
49
42
48
41
49
42
49
42
48
41
49
42


In [35]:
version_to_df['v2']

,0
0,[[AdvancedFormat]]
1,[[Block:MC Block]]
2,[[Question:MC:SingleAnswer:Horizontal]]
3,<strong>Q1.</strong><br>
4,<strong>Text 1:</strong> We advocate for homo...
...,...
472,Consistent
473,Indirect inconsistency
474,Factual inconsistency
475,Surface contradiction


In [5]:
df_final = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\Annotation_Round - Sheet_Round_1.csv", header=None)

In [7]:
df_final = df_final[250:]

In [8]:
df_final

,0,1,2,3
250,The residence requirement significantly restri...,Asylum seekers should remain within the distri...,1.0,Surface
251,Snowmaking systems may seem attractive in the ...,We fully support the continued funding of mode...,2.0,NaN
252,Museums should be less regulated in terms of t...,Entrance to state museums must be free for eve...,3.0,NaN
253,"The bargaining parties, unions, and employers ...",Companies with state contracts must be require...,4.0,NaN
254,Minimum wage should not be differentiated base...,Companies with state contracts must be require...,5.0,NaN
...,...,...,...,...
345,Congestion charge places additional burdens on...,We support the introduction of a congestion ch...,96.0,NaN
346,Human rights violation taking place in the Mag...,We support initiatives to classify the Maghreb...,97.0,NaN
347,"""The fireworks get on the nerves of both human...",We support allowing the lighting of private Ne...,98.0,NaN
348,"""The construction and operation of wind turbin...",We support the continued construction of wind ...,99.0,NaN


In [12]:
df_final[1][348]

'We support the continued construction of wind turbines in forested areas.'

In [13]:
from openai import OpenAI

In [22]:
import time
import random

In [18]:
def biased_random():
    return 1 if random.random() < 2/3 else 0

In [23]:
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")
paraphrase_id = 'asst_F6crXcvUsjq5obDKyJMY6Y4l'

In [24]:
def paraphrase_support(text):

    statement = text
  
    # Create a thread with a message.
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                # Update this with the query you want to use.
                "content": statement,
            }
        ]
    )
    # Submit the thread to the assistant (as a new run).
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=paraphrase_id)

    # Wait for run to complete.
    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(1)
    # else:
    #     print(f"🏁 Run Completed!")

    # Get the latest message from the thread.
    message_response = client.beta.threads.messages.list(thread_id=thread.id)
    messages = message_response.data

    # Print the latest message.
    latest_message = messages[0]
    return latest_message.content[0].text.value
        # except:
        #     print(f"Run went wrong")
        #     return None

In [25]:
for i in tqdm(range(250, 350, 1)):
    my_text = df_final[1][i]
    if "support" in my_text:
        res = biased_random()
        if res == 1:
            print("Paraphrasing...")
            df_final[1][i] = paraphrase_support(my_text)
        

  0%|          | 0/100 [00:00<?, ?it/s]

Paraphrasing...


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_170204\3423503589.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[1][i] = paraphrase_support(my_text)
  2%|▏         | 2/100 [00:04<03:42,  2.27s/it]

Paraphrasing...


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_170204\3423503589.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[1][i] = paraphrase_support(my_text)
 10%|█         | 10/100 [00:08<01:07,  1.32it/s]

...

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_170204\3423503589.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[1][i] = paraphrase_support(my_text)
 96%|█████████▌| 96/100 [02:11<00:07,  1.98s/it]

Paraphrasing...


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_170204\3423503589.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[1][i] = paraphrase_support(my_text)
 97%|█████████▋| 97/100 [02:15<00:06,  2.21s/it]

Paraphrasing...


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_170204\3423503589.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[1][i] = paraphrase_support(my_text)
100%|██████████| 100/100 [02:19<00:00,  1.39s/it]


In [28]:
df_final.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\surface_paraphrase.csv", index=False)